In [1]:
!pip install opencv-python
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from tensorflow.keras import models, layers, activations, \
optimizers, utils, losses, initializers, metrics, callbacks
import os
import cv2 as cv

In [4]:
folder_path = 'testing'
epochs = 100
batch_size = 32
patience = 5
learning_rate = 0.001
model_path = f'checkpoints/{folder_path}/test_model5.keras'
exists = os.path.exists(model_path)

# Carrega modelo se já existir um checkpoint, caso contrário, o cria.
if exists:
    model = models.load_model(model_path)
else: 
    model = models.Sequential([
        layers.Resizing(32, 32),
        layers.Rescaling(1.0/255),
        layers.BatchNormalization(axis=1),
        layers.RandomRotation((-0.05, 0.05)),
        layers.Conv2D(32, (3, 3),
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
            ),
        # layers.MaxPooling2D((2, 2)),
        layers.Conv2D(32, (3, 3),
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        # layers.MaxPooling2D((2, 2)), # tirar dps
        layers.Conv2D(32, (3, 3),
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        # layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128,
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        layers.Dense(64,
            activation = 'relu',
            kernel_initializer = initializers.RandomNormal()
        ),
        layers.Dense(62,
            activation = 'sigmoid',
            kernel_initializer = initializers.RandomNormal()
        )
        ])

if exists:
    model.summary()
else:
    model.compile(
    optimizer = optimizers.Adam(
    learning_rate = learning_rate
),
    loss = losses.SparseCategoricalCrossentropy(), # trocar 
    metrics = [ 'accuracy' ]
)
    
train = utils.image_dataset_from_directory(
    "organizedData",
    validation_split= 0.2,
    subset= "training",
    seed= 123,
    shuffle= True,
    image_size= (1200, 900),
    batch_size= batch_size
    )

test = utils.image_dataset_from_directory(
    "organizedData",
    validation_split= 0.2,
    subset= "validation",
    seed= 123,
    shuffle= True,
    image_size= (1200, 900),
    batch_size= batch_size
    )

model.fit(train,
    epochs = epochs,
    validation_data = test,
    callbacks= [
        callbacks.EarlyStopping(
            monitor = 'val_loss',
            patience = patience,
            verbose = 1
        ),
        callbacks.ModelCheckpoint(
            filepath = model_path,
            save_weights_only = False,
            monitor = 'loss',
            mode = 'min',
            save_best_only = True
        ),
        # callbacks.LearningRateScheduler(
        #     lambda epoch, lr : 1e-3 * 10 ** -(epoch / 20)
        # )
    ]
)

Found 10230 files belonging to 62 classes.
Using 8184 files for training.
Found 10230 files belonging to 62 classes.
Using 2046 files for validation.
Epoch 1/100
256/256 [==============================] - 32s 117ms/step - loss: 3.5515 - accuracy: 0.1113 - val_loss: 2.5975 - val_accuracy: 0.2952
Epoch 2/100
256/256 [==============================] - 32s 125ms/step - loss: 2.3205 - accuracy: 0.3420 - val_loss: 1.4815 - val_accuracy: 0.5376
Epoch 3/100
256/256 [==============================] - 32s 123ms/step - loss: 1.6992 - accuracy: 0.4961 - val_loss: 1.0754 - val_accuracy: 0.6569
Epoch 4/100
256/256 [==============================] - 32s 123ms/step - loss: 1.4109 - accuracy: 0.5595 - val_loss: 0.9725 - val_accuracy: 0.6867
Epoch 5/100
256/256 [==============================] - 32s 124ms/step - loss: 1.2566 - accuracy: 0.6006 - val_loss: 0.8734 - val_accuracy: 0.7067
Epoch 6/100
256/256 [==============================] - 32s 124ms/step - loss: 1.1508 - accuracy: 0.6295 - val_loss: 0.79